In [65]:
import requests
import config
import json
import pandas as pd
#from requests.auth import HTTPBasicAuth

username = 'KseniaBuk'
api_username = config.username
password = config.password 

url = f'https://api.ravelry.com/people/{username}/favorites/list.json'
print(url)

https://api.ravelry.com/people/KseniaBuk/favorites/list.json


In [66]:
payload = {
    'page': str(1),
    'page_size': 25
}
response = requests.get(url, params=payload, auth=(api_username, password))

In [67]:
if response.status_code == 200:
    data = response.json()

pages = data['paginator']['page_count']
favorites = data.get('favorites', [])
favorites_trunc = []
p_id = []

for f in favorites:
    fn = {
        'name' : f['favorited']['name'], 
        'created_at' : f['created_at'], 
        'id' : f['favorited']['id'],
        'permalink' : f['favorited']['permalink'],
        'page' : 1
    }
    favorites_trunc.append(fn)
    p_id.append(f['favorited']['id'])

In [68]:
print(pages)

10


In [69]:
for i in range(2, pages  + 1):
    payload = {
        'page': str(i),
        'page_size': 25
    }
    response = requests.get(url, params=payload, auth=(api_username, password))
    data = response.json()
    favorites = data.get('favorites', [])
    for f in favorites:
        fn = {
            'name' : f['favorited']['name'], 
            'created_at' : f['created_at'], 
            'id' : f['favorited']['id'],
            'permalink' : f['favorited']['permalink'],
            'page' : i
        }
        favorites_trunc.append(fn)
        p_id.append(f['favorited']['id'])

In [70]:
favorites_df = pd.json_normalize(favorites_trunc)
favorites_df.sort_values('created_at', ascending = False).head()

,name,created_at,id,permalink,page
0,Turnaround,2025/09/05 09:51:11 -0400,7459516,turnaround-2,1
1,Hairst Hap Hood,2025/09/03 11:08:08 -0400,7457950,hairst-hap-hood,1
2,Berlin Baby Tee,2025/08/31 15:39:40 -0400,7341902,berlin-baby-tee,1
3,Crossandra Sweater Adult,2025/08/31 15:35:38 -0400,7371600,crossandra-sweater-adult,1
4,Deima’s Crochet Cardi,2025/08/31 15:34:10 -0400,7424682,deimas-crochet-cardi,1


In [71]:
patterns = []
for i in p_id: 
    url = f'https://api.ravelry.com/patterns/{i}.json'
    response = requests.get(url, auth=(api_username, password))
    if response.status_code == 200:
        data = response.json()
        patterns.append(data)

In [72]:
patterns_trunc = []
for p in patterns:
    pn = {
        'p_id' : p['pattern']['id'], 
        'difficulty_average' : p['pattern']['difficulty_average'],
        'difficulty_count' : p['pattern']['difficulty_count'],
        'gauge' : p['pattern']['gauge'],
        'gauge_divisor' : p['pattern']['gauge_divisor'],
        'gauge_pattern' : p['pattern']['gauge_pattern'],
        'rating_average' : p['pattern']['rating_average'],
        'rating_count' : p['pattern']['rating_count'],
        'row_gauge' : p['pattern']['row_gauge'],
        'url' : p['pattern']['url'],
        'yardage' : p['pattern']['yardage'],
        'yardage_max' : p['pattern']['yardage_max'],
        'yarn_weight_description' : p['pattern']['yarn_weight_description'],
        'yardage_description' : p['pattern']['yardage_description'],
        'knit_gauge' : p['pattern'].get('yarn_weight', {}).get('knit_gauge'),
        'name' : p['pattern'].get('yarn_weight', {}).get('name'),
        'ply' : p['pattern'].get('yarn_weight', {}).get('ply'),
        'wpi' : p['pattern'].get('yarn_weight', {}).get('wpi')
    }
    patterns_trunc.append(pn)

In [73]:
patterns_df = pd.json_normalize(patterns_trunc)
patterns_df.head()

,p_id,difficulty_average,difficulty_count,gauge,gauge_divisor,gauge_pattern,rating_average,rating_count,row_gauge,url,yardage,yardage_max,yarn_weight_description,yardage_description,knit_gauge,name,ply,wpi
0,7459516,4.000000,5.0,25.0,4.0,stockinette Stitch,5.000000,6.0,32.0,,656.0,1039.0,Fingering (14 wpi),656 - 1039 yards,28,Fingering,4,14
1,7457950,0.000000,NaN,23.0,4.0,garter stitch,0.000000,NaN,38.0,,470.0,560.0,Fingering (14 wpi),470 - 560 yards,28,Fingering,4,14
2,7341902,2.444444,9.0,NaN,4.0,,3.888889,9.0,NaN,https://ko-fi.com/s/492fc52ca1,NaN,NaN,Any gauge - designed for any gauge,yards,None,Any gauge,None,None
3,7371600,3.600000,5.0,14.0,4.0,Garter stitch,4.857143,7.0,24.0,,1772.0,2953.0,Bulky (7 wpi),1772 - 2953 yards,14-15,Bulky,12,7
4,7424682,3.000000,3.0,14.0,4.0,stockinette stitch,4.666667,3.0,18.0,https://deimaknitwear.com/products/deimas-croc...,919.0,1608.0,Aran (8 wpi),919 - 1608 yards,18,Aran,10,8


In [74]:
favorites_df.to_excel('favorites.xlsx')
patterns_df.to_excel('patterns.xlsx')